In [14]:
using CSV
import Pkg
Pkg.add("JSON3")
using JSON3
using DataFrames


# Read the Locations CSV file
df = CSV.read("Locations_Capabilities.csv", DataFrame)

# Function to parse JSON strings into Julia objects
function parse_json_string(str)
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    return JSON3.read(str)
end

# Process each location's data
locations_data = Dict()

for row in eachrow(df)
    location_id = row.Location_ID
    
    # Process workers data
    workers = parse_json_string(row.Workers)
    
    # Process machines data
    machines = parse_json_string(row.Machines)
    
    # Process inventory resources
    inventory = parse_json_string(row.Inventory_Resources)
    
    # Store all data for this location
    locations_data[location_id] = Dict(
        "workers" => workers,
        "machines" => machines,
        "inventory" => inventory
    )
end

# Create variables for each location
const L1_data = locations_data["L1"]
const L2_data = locations_data["L2"]

# Extract specific components for easier access
const L1_workers = L1_data["workers"]
const L1_machines = L1_data["machines"]
const L1_inventory = L1_data["inventory"]

const L2_workers = L2_data["workers"]
const L2_machines = L2_data["machines"]
const L2_inventory = L2_data["inventory"]

# Example of how to access the data:
# L1_workers[1]["Worker_ID"]  # Gets the ID of the first worker at L1
L1_machines[1]["Machine_ID"]  # Gets the ID of the first machine at L1
# L1_inventory["Steel"]  # Gets the amount of steel at L1

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


"L1_M1"

In [15]:
using CSV
using JSON3
using DataFrames

# Read the CSV file
df = CSV.read("Projects_Data_with_Stage_Durations.csv", DataFrame)

# Function to clean and parse JSON strings into Julia objects
function parse_json_string(str)
    # Clean the string: remove newlines and extra spaces
    str = replace(str, r"\s+" => " ")  # Replace multiple spaces with single space
    str = replace(str, "\n" => "")     # Remove newlines
    str = strip(str)                   # Remove leading/trailing whitespace
    
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    
    try
        return JSON3.read(str)
    catch e
        println("Error parsing JSON string: ", str)
        rethrow(e)
    end
end

# Process each project's data
projects_data = Dict()

for row in eachrow(df)
    project_id = strip(row.Project_ID)
    
    # Process all JSON fields
    total_resources = parse_json_string(row.Total_Resources)
    workers_req = parse_json_string(row.Workers_Requirement)
    machines_req = parse_json_string(row.Machines_Requirement)
    stages = parse_json_string(row.Stages)
    
    # Store all data for this project
    projects_data[project_id] = Dict(
        "total_resources" => total_resources,
        "workers_requirement" => workers_req,
        "machines_requirement" => machines_req,
        "deadline" => row.Deadline,
        "customer_priority" => row.Customer_Priority,
        "stages" => stages,
        "revenue" => row.Revenue
    )
end

# Create helper functions
function get_project_data(project_id)
    return projects_data[project_id]
end

function get_all_project_ids()
    return collect(keys(projects_data))
end

function get_project_stages(project_id)
    return projects_data[project_id]["stages"]
end

function get_project_resources(project_id)
    return projects_data[project_id]["total_resources"]
end

function get_project_workers(project_id)
    return projects_data[project_id]["workers_requirement"]
end

function get_project_machines(project_id)
    return projects_data[project_id]["machines_requirement"]
end

function get_project_deadline(project_id)
    return projects_data[project_id]["deadline"]
end

function get_project_priority(project_id)
    return projects_data[project_id]["customer_priority"]
end

function get_project_revenue(project_id)
    return projects_data[project_id]["revenue"]
end
 

# Print some data to verify it worked
first_project_id = first(get_all_project_ids())
println("First project data:")
println("Project ID: ", first_project_id)
println("Deadline: ", get_project_deadline(first_project_id))
println("Priority: ", get_project_priority(first_project_id))
println("Revenue: ", get_project_revenue(first_project_id))
println("Total Resources: ", get_project_resources(first_project_id))
println("First Stage: ", get_project_stages(first_project_id)[1])

First project data:
Project ID: P13
Deadline: 66
Priority: 1
Revenue: 16000
Total Resources: {
   "Plastic": 66,
     "Steel": 51,
     "Paint": 35
}
First Stage: {
           "Stage_ID": 1,
    "Machines_Needed": [
                         "Inspector"
                       ],
     "Workers_Needed": [
                         "Painter",
                         "Inspector"
                       ],
   "Resources_Needed": {
                          "Plastic": 47,
                            "Steel": 27,
                            "Paint": 13
                       },
      "Duration_days": 8
}


In [ ]:
m = Model(Gurobi.Optimizer)

@variable(m,p[1:n] >= 0)
@variable(m,s[1:n] >= 0)
@variable(m,x[1:n] >= 0)

@objective(m, Max, sum((get_project_revenue(project_id)) + (get_project_deadline[project_id]-t[project_id][-1]))*z[project_id] for project_id in get_all_project_ids())


@constraint(m, sum(m[machine_id][1:end-1][location_id] for machine_id in get_all_machine_ids()) <= 1) #machine M can only be used by one project p at a time t at a location l
@constraint(m, sum(w[worker_id][1:end-1][location_id] for worker_id in get_all_worker_ids()) <= 1) #worker W can only be used by one project p at a time t at a location l
@constraint(m, (x-b)'Q*(x-b) <= 100)

optimize!(m)